In [1]:
from google.colab import drive
drive.mount('/content/drive',  force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install transformers

     |████████████████████████████████| 778kB 13.0MB/s 
     |████████████████████████████████| 3.0MB 43.1MB/s 
     |████████████████████████████████| 1.1MB 45.5MB/s 
     |████████████████████████████████| 890kB 45.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=aca355c3f795ac06a0da3a7ab9fcf122b99d1dd9c8f88fda581748d03bbce013
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [3]:
import transformers
import numpy as np
import json
import pandas as pd
import re
import io

In [4]:
from tensorflow.keras import models, layers, preprocessing as kprocessing
from tensorflow.keras import backend as K
import sklearn.metrics as metrics

In [5]:
nlp_bert = transformers.TFBertModel.from_pretrained('distilbert-base-uncased')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing TFBertModel: ['activation_13', 'vocab_projector', 'vocab_layer_norm', 'distilbert', 'vocab_transform']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of TFBertModel were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['bert']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
tokenizer = transformers.AutoTokenizer.from_pretrained('distilbert-base-uncased', do_lower_case=True)

# Generating Metadata

In [15]:
def get_metadata_features(i):
    print("Inside get_metadata_features")
    """
    Obtain the dataset
    """
    # Extracted the text for nlp embeddings  
    df_X_train= pd.read_pickle('/content/drive/My Drive/Python Notebook/SCS_CONVEX/data-files/search/X_train_' + str(i) + '.pkl')
    #print(df_X_train.head())
    df_X_test= pd.read_pickle('/content/drive/My Drive/Python Notebook/SCS_CONVEX/data-files/search/X_test_' + str(i) + '.pkl')
    df_Y_train=pd.read_pickle('/content/drive/My Drive/Python Notebook/SCS_CONVEX/data-files/search/Y_train_' + str(i) + '.pkl')
    df_Y_test=pd.read_pickle('/content/drive/My Drive/Python Notebook/SCS_CONVEX/data-files/search/Y_test_' + str(i) + '.pkl')
    #print(df_X_train.shape) #(835, 5)
    #print(df_Y_train.shape) #(835, 1)
    #print(df_X_test.shape) #(209, 5)
    #print(df_Y_test.shape) #(209, 1)
    
    X_train_meta = df_X_train.drop(['Previous_User_Utterance'], axis=1)
    X_test_meta = df_X_test.drop(['Previous_User_Utterance'], axis=1)
    
    #print(X_train_meta['Previous_Speech_Act'].unique())
    #print(X_train_meta['Previous_Search_Act'].unique())
    
    return X_train_meta, X_test_meta, df_Y_train, df_Y_test

# Attention Layer

In [8]:
from keras.engine.topology import Layer
class AttentionLayer(Layer):
    
    def __init__(self,attention_dim=100,return_coefficients=False,**kwargs):
        # Initializer 
        self.supports_masking = True
        self.return_coefficients = return_coefficients
        self.init = initializers.get('glorot_uniform') # initializes values with uniform distribution
        self.attention_dim = attention_dim
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Builds all weights
        # W = Weight matrix, b = bias vector, u = context vector
        assert len(input_shape) == 3
        self.W = K.variable(self.init((input_shape[-1], self.attention_dim)),name='W')
        self.b = K.variable(self.init((self.attention_dim, )),name='b')
        self.u = K.variable(self.init((self.attention_dim, 1)),name='u')
        self.trainable_weight = [self.W, self.b, self.u]

        super(AttentionLayer, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, hit, mask=None):
        # Here, the actual calculation is done
        uit = K.bias_add(K.dot(hit, self.W),self.b)
        uit = K.tanh(uit)
        
        ait = K.dot(uit, self.u)
        ait = K.squeeze(ait, -1)
        ait = K.exp(ait)
        
        if mask is not None:
            ait *= K.cast(mask, K.floatx())

        ait /= K.cast(K.sum(ait, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        ait = K.expand_dims(ait)
        weighted_input = hit * ait
        
        if self.return_coefficients:
            return [K.sum(weighted_input, axis=1), ait]
        else:
            return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        if self.return_coefficients:
            return [(input_shape[0], input_shape[-1]), (input_shape[0], input_shape[-1], 1)]
        else:
            return input_shape[0], input_shape[-1]

# BERT feature generate

In [9]:
def load_files(i):

    df_X_train= pd.read_pickle('/content/drive/My Drive/Python Notebook/SCS_CONVEX/data-files/search/X_train_' + str(i) + '.pkl')
    df_X_test= pd.read_pickle('/content/drive/My Drive/Python Notebook/SCS_CONVEX/data-files/search/X_test_' + str(i) + '.pkl')
    df_Y_train=pd.read_pickle('/content/drive/My Drive/Python Notebook/SCS_CONVEX/data-files/search/Y_train_' + str(i) + '.pkl')
    df_Y_test=pd.read_pickle('/content/drive/My Drive/Python Notebook/SCS_CONVEX/data-files/search/Y_test_' + str(i) + '.pkl')
    df_train = pd.concat([df_X_train,df_Y_train], axis=1)
    df_test = pd.concat([df_X_test,df_Y_test], axis=1)

    #print(list(df_train.columns))

    df_train = df_train[["Search_acts","Previous_User_Utterance"]]
    df_train = df_train.rename(columns={"Search_acts":"y", "Previous_User_Utterance":"text"})

    

    df_test = df_test[["Search_acts","Previous_User_Utterance"]]
    df_test = df_test.rename(columns={"Search_acts":"y", "Previous_User_Utterance":"text"})

    return df_train,df_test,df_Y_train,df_Y_test

In [10]:
def generate_maskid(training, test):
  #print(training['text'])
  corpus_train = training['text']
  corpus_test = test['text']
  #the length of the feature vector is 150
  maxlen = 150

  #add special tokens
  maxqnans = np.int((maxlen-20)/2)
  corpus_tokenized_train = ["[CLS] "+
              " ".join(tokenizer.tokenize(re.sub(r'[^\w\s]+|\n', '', 
              str(txt).lower().strip()))[:maxqnans])+
              " [SEP] " for txt in corpus_train]
  corpus_tokenized_test = ["[CLS] "+
              " ".join(tokenizer.tokenize(re.sub(r'[^\w\s]+|\n', '', 
              str(txt).lower().strip()))[:maxqnans])+
              " [SEP] " for txt in corpus_test]
  #generate masks
  masks_train = [[1]*len(txt.split(" ")) + [0]*(maxlen - len(txt.split(" "))) for txt in corpus_tokenized_train]
  masks_test = [[1]*len(txt.split(" ")) + [0]*(maxlen - len(txt.split(" "))) for txt in corpus_tokenized_test]

  #padding
  txt2seq_train = [txt + " [PAD]"*(maxlen-len(txt.split(" "))-2) if len(txt.split(" ")) != (maxlen) else txt for txt in corpus_tokenized_train]
  txt2seq_test = [txt + " [PAD]"*(maxlen-len(txt.split(" "))-2) if len(txt.split(" ")) != (maxlen) else txt for txt in corpus_tokenized_test]

  #generate idx
  idx_train = [tokenizer.encode(seq.split(" ")) for seq in txt2seq_train]
  idx_test = [tokenizer.encode(seq.split(" ")) for seq in txt2seq_test]      

  ## feature matrix
  X_train = [np.asarray(idx_train, dtype='int32'), 
            np.asarray(masks_train, dtype='int32')]
  X_test = [np.asarray(idx_test, dtype='int32'), 
            np.asarray(masks_test, dtype='int32')]
  

  idx_train = np.asarray(idx_train, dtype='int32')
  masks_train = np.asarray(masks_train, dtype='int32')
  idx_test = np.asarray(idx_test, dtype='int32')
  masks_test = np.asarray(masks_test, dtype='int32')

  #print("txt: ", training["text"].iloc[0])
  #print("tokenized:", [tokenizer.convert_ids_to_tokens(idx) for idx in X_train[0][i].tolist()])
  #print("idx: ", X_train[0][i])
  #print("mask: ", X_train[1][i])
  return idx_train, masks_train, idx_test, masks_test


# Deep Neural Model Creation

In [11]:
def create_bilstm_meta_bert_channels(reshaped_data_meta):
    
    idx = Input((150), dtype="int32", name="input_idx")
    masks = Input((150), dtype="int32", name="input_masks")
    ## pre-trained bert with config
    config = transformers.DistilBertConfig(dropout=0.2,attention_dropout=0.2)
    config.output_hidden_states = False
    nlp_bert = transformers.TFDistilBertModel.from_pretrained('distilbert-base-uncased', config=config)
    bert_out = nlp_bert(idx, attention_mask=masks)[0]
    ## fine-tuning
    x = GlobalAveragePooling1D()(bert_out)
    x = Dense(64, activation="relu")(x)
    #x = layers.Dense(32, activation="relu")(x)
    y_out = Dense(4, activation='softmax')(x)
    
    
    
    samples,timesteps,features = reshaped_data_meta.shape
    inp_meta = Input(shape=(timesteps,features), name='input_meta')
    # lstm needs (samples,timesteps,features) tensor as the input
    x1 = Bidirectional(LSTM(BATCH_SIZE, return_sequences=True, dropout=0.25, recurrent_dropout=0.1))(inp_meta) 
    x1, sent_coeffs1 = AttentionLayer(features,return_coefficients=True,name='sent_attention1')(x1)
    #x1 = GlobalMaxPool1D()(x1)
    #x1 = Dense(100, activation="relu")(x1)
    x1 = Dropout(0.25)(x1)
    x1 = Dense(4, activation="softmax")(x1) #output layer  
    
    
      
    
    """
    Merging outputs of nlp and meta
    """
    merge_meta_bert = concatenate([x1, y_out])


    output_meta_bert = Dense(4, activation="softmax")(merge_meta_bert)
    model_meta_bert_combined = Model(inputs=[inp_meta, idx, masks], outputs=output_meta_bert)
    
    return model_meta_bert_combined

# META BERT model Training

In [16]:
def execute_bilstm_meta_bert_channel(i):

  """
  meta data
  """
  df_train_meta, df_test_meta, df_Y_train_meta, df_Y_test_meta = get_metadata_features(i)
  train_meta = df_train_meta.values
  test_meta = df_test_meta.values
      
  timestamp = 1 #number of successive sequences combined
      
  r, c = df_train_meta.shape
  staggering = r - timestamp + 1 # final number of instances generated 
  X_train_reshaped_meta = np.concatenate([train_meta[x:x+timestamp,:] for x in range(r-timestamp+1)])
  X_train_reshaped_meta = X_train_reshaped_meta.reshape(staggering, timestamp, c) # c is the number of features
    
  r2, c2 = df_test_meta.shape
  staggering2 = r2 - timestamp + 1 # final number of instances generated 
  X_test_reshaped_meta = np.concatenate([test_meta[x:x+timestamp,:] for x in range(r2-timestamp+1)])
  X_test_reshaped_meta = X_test_reshaped_meta.reshape(staggering2, timestamp, c2) # c is the number of features
      
  print("Meta data\n------------------")
  print("Training set\n------------------")
  print(X_train_reshaped_meta)
  print(X_train_reshaped_meta.shape)
      
  print("Meta data\n------------------")
  print("Test set\n------------------")
  print(X_test_reshaped_meta)
  print(X_test_reshaped_meta.shape)
      
  df_Y_train_meta = df_Y_train_meta.iloc[timestamp-1:,]
  df_Y_test_meta = df_Y_test_meta.iloc[timestamp-1:,]
  
  
  print("Output Labels\n------------------")
  print(df_Y_train_meta.shape, df_Y_test_meta.shape)

  """
  bert data
  """
  df_train_bert,df_test_bert,df_Y_train_bert,df_Y_test_bert = load_files(i)
  y_train_bert = df_Y_train_bert['Search_acts'].to_list()
  y_test_bert = df_Y_test_bert['Search_acts'].to_list()
  X_train_id, X_train_mask, X_test_id, X_test_mask = generate_maskid(df_train_bert,df_test_bert)
      
  model_meta_bert = create_bilstm_meta_bert_channels(X_train_reshaped_meta)


  print("\n\n THESE ARE THE LAYERS")
  for layer in model_meta_bert.layers[:5]:
    if(layer.name == 'input_meta' or layer.name == 'bidirectional'):
      layer.trainable = True
    else:
      layer.trainable = False

  model_meta_bert.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['binary_accuracy', 'categorical_accuracy'])
  model_meta_bert.summary()


  """
  encoding the output labels
  """
      
  encoder = LabelEncoder()
  encoder.fit(y_train_bert)
  training_op_labels_encoded = encoder.transform(y_train_bert)
  test_op_labels_encoded = encoder.transform(y_test_bert)
  print("Output Labels\n-------------------")
  print(training_op_labels_encoded.shape, test_op_labels_encoded.shape)
      
  """
  converting the output labels to one-hot form
  """
  training_op_labels_onehot= np_utils.to_categorical(training_op_labels_encoded)
  test_op_labels_onehot = np_utils.to_categorical(test_op_labels_encoded)

  print(training_op_labels_onehot.shape, len(training_op_labels_onehot))
  print(test_op_labels_onehot.shape, len(test_op_labels_onehot))


  X_train_reshaped_meta = np.asarray(X_train_reshaped_meta, dtype='float32')
  #training_op_labels_onehot = tf.convert_to_tensor(training_op_labels_onehot)
  
  
  model_meta_bert_train = model_meta_bert.fit(x=[X_train_reshaped_meta,X_train_id,X_train_mask], y=training_op_labels_onehot,batch_size=BATCH_SIZE, epochs=EPOCHS,  validation_split=0.1)
  # load model if required 
  # compile model


  model_meta_bert.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['binary_accuracy', 'categorical_accuracy'])
      
  # evaluate model
  print(model_meta_bert.metrics_names)
  #change the input_

  #loss, binary_accuracy, categorical_accuracy = model_nlp_bert.evaluate(x=[X_test_reshaped_nlp,X_test_id,X_test_mask], y=test_op_labels_onehot,batch_size=BATCH_SIZE,verbose=1)
      
  #print(loss, binary_accuracy, categorical_accuracy)
      

  """
  predict the probabilty of output classes
  and pick the best one
  """
  #change the input_

  X_test_reshaped_meta = np.asarray(X_test_reshaped_meta, dtype='float32')
  prediction_vector = model_meta_bert.predict(x=[X_test_reshaped_meta,X_test_id,X_test_mask], \
                            batch_size=BATCH_SIZE,\
                            verbose=1)
  predicted_classes = np.argmax(prediction_vector, axis=1)
  original_classes = np.argmax(test_op_labels_onehot, axis=1)
  accuracy = metrics.accuracy_score(original_classes, predicted_classes)

  print("ACCURACY:")
  print(accuracy)
      
  """
  # verification of correctness:
  total_correct = sum(original_classes == predicted_classes)
  print("Total number of correct predictions:",total_correct)
  print("Accuracy:",total_correct/len(test_op_labels_onehot))
  acc = np.sum(conf_mat.diagonal()) / np.sum(conf_mat)
  print('Overall accuracy: {} %'.format(acc*100))
  """
  conf_mat = confusion_matrix(predicted_classes, original_classes)
  print("\n", conf_mat, "\n")
      
  return predicted_classes, accuracy, conf_mat

# Main Method and Library Imports

In [17]:
"""
import tensorflow as tf
tf_config = tf.compat.v1.ConfigProto()
tf_config.gpu_options.allow_growth = True
tf_config.gpu_options.per_process_gpu_memory_fraction = 0.9
tf_config.allow_soft_placement = True
"""

import numpy as np 
import pandas as pd
from numpy import array

import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Bidirectional, GlobalMaxPool1D, GlobalAveragePooling1D
from keras.layers.merge import concatenate
from keras.models import Model, model_from_json
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.utils import np_utils, to_categorical
from keras.engine.topology import Layer
from keras import backend as K

from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

# checkpoint
from keras.callbacks import ModelCheckpoint

from tensorflow.keras.layers import Attention

"""
Required for NLP model
"""
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()   

# set parameters for word embeddings
embed_size = 100 # how big is each word vector
vocab_size = 25000 # how many unique words to use (i.e num rows in embedding vector) max
input_length = 100 # max number of words in the input 

#set parameters for bilstm
BATCH_SIZE=32
EPOCHS=1 #300

#EMBEDDING_FILE='glove.6B.100d.txt'    

file = open('/content/drive/My Drive/Python Notebook/SCS_CONVEX/output/Search Tasks/meta-bert-attn/bilstm_meta_bert_attn_op.txt','w') #overwrites previous
file.close()

if __name__== "__main__":
    df_prediction = pd.DataFrame()
    df_accuracy =  pd.DataFrame()
    file = open('/content/drive/My Drive/Python Notebook/SCS_CONVEX/output/Search Tasks/meta-bert-attn/bilstm_meta_bert_attn_op.txt','a') #append mode 
    """
    Change the range before executing
    """
    for i in range(5,11):
        outputname = 'meta_bert_attn'+ str(i)        
        predictions, acc, conf_matrix = execute_bilstm_meta_bert_channel(i)
        df_prediction[outputname] = predictions
        df_accuracy[i] = [acc]
        file.write("\nIteration:" + str(i) + "\nCategorical Accuracy:" + str(acc) + 
                    "\nConfusion Matrix:\n" + str(conf_matrix) + "\n\n")
        df_prediction.to_csv('/content/drive/My Drive/Python Notebook/SCS_CONVEX/output/Search Tasks/meta-bert-attn/predictions_bilstm_meta_bert_attn_' + str(i) + '.csv')    
        df_accuracy.to_csv('/content/drive/My Drive/Python Notebook/SCS_CONVEX/output/Search Tasks/meta-bert-attn/accuracy_bilstm_meta_bert_attn_' + str(i) + '.csv')    
    
    df_prediction.to_csv('/content/drive/My Drive/Python Notebook/SCS_CONVEX/output/Search Tasks/meta-bert-attn/predictions_bilstm_meta_bert_attn.csv')    
    df_accuracy.to_csv('/content/drive/My Drive/Python Notebook/SCS_CONVEX/output/Search Tasks/meta-bert-attn/accuracy_bilstm_meta_bert_attn.csv')    
    file.close()

Inside get_metadata_features
Meta data
------------------
Training set
------------------
[[[0.0 46.0 0 ... 10 2 1]]

 [[0.0 12.0 0 ... 1 2 3]]

 [[0.0 4.0 0 ... 10 2 3]]

 ...

 [[0.0 5.0 0 ... 1 2 4]]

 [[0.0 24.0 0 ... 1 2 1]]

 [[0.0 15.0 0 ... 1 2 0]]]
(406, 1, 8)
Meta data
------------------
Test set
------------------
[[[0.0 31.0 0 0 1 10 2 3]]

 [[0.0 13.0 0 0 2 1 2 2]]

 [[0.0 5.0 0 1 1 10 10 3]]

 [[0.0 7.0 0 0 0 1 2 2]]

 [[0.0 7.0 0 1 2 1 1 3]]

 [[0.0 7.0 0 0 2 12 12 3]]

 [[0.0 44.0 0 1 1 10 2 1]]

 [[0.0 29.0 0 0 1 1 2 2]]

 [[0.0 8.0 0 1 1 10 2 1]]

 [[0.0 10.0 0 0 2 1 2 1]]

 [[0.0 47.0 0 0 1 1 2 1]]

 [[0.0 23.0 0 0 2 1 2 1]]

 [[0.0 10.0 0 1 1 10 2 1]]

 [[0.0 43.0 0 0 2 1 11 3]]

 [[0.0 4.0 0 1 1 10 10 3]]

 [[0.0 3.0 0 0 0 1 2 3]]

 [[0.0 21.0 0 0 2 1 2 1]]

 [[0.0 10.0 0 1 1 10 2 1]]

 [[0.0 10.0 0 1 2 12 12 3]]

 [[0.0 13.0 0 1 2 6 6 1]]

 [[0.0 5.0 0 0 0 1 2 3]]

 [[0.0 5.0 0 1 1 1 1 2]]

 [[0.0 25.0 0 0 0 1 2 4]]

 [[0.0 10.0 0 1 1 10 2 1]]

 [[0.0 7.0 0 1 2 1 

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['activation_13', 'vocab_transform', 'vocab_layer_norm', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFDistilBertModel for predictions without further training.




 THESE ARE THE LAYERS
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_meta (InputLayer)         [(None, 1, 8)]       0                                            
__________________________________________________________________________________________________
input_idx (InputLayer)          [(None, 150)]        0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 150)]        0                                            
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 1, 64)        10496       input_meta[0][0]                 
_______________________________________________________________

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['activation_13', 'vocab_transform', 'vocab_layer_norm', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFDistilBertModel for predictions without further training.




 THESE ARE THE LAYERS
Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_meta (InputLayer)         [(None, 1, 8)]       0                                            
__________________________________________________________________________________________________
input_idx (InputLayer)          [(None, 150)]        0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 150)]        0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 1, 64)        10496       input_meta[0][0]                 
_______________________________________________________________

...



 THESE ARE THE LAYERS
Model: "functional_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_meta (InputLayer)         [(None, 1, 8)]       0                                            
__________________________________________________________________________________________________
input_idx (InputLayer)          [(None, 150)]        0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 150)]        0                                            
__________________________________________________________________________________________________
bidirectional_3 (Bidirectional) (None, 1, 64)        10496       input_meta[0][0]                 
_______________________________________________________________

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['activation_13', 'vocab_transform', 'vocab_layer_norm', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFDistilBertModel for predictions without further training.




 THESE ARE THE LAYERS
Model: "functional_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_meta (InputLayer)         [(None, 1, 8)]       0                                            
__________________________________________________________________________________________________
input_idx (InputLayer)          [(None, 150)]        0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 150)]        0                                            
__________________________________________________________________________________________________
bidirectional_4 (Bidirectional) (None, 1, 64)        10496       input_meta[0][0]                 
_______________________________________________________________

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['activation_13', 'vocab_transform', 'vocab_layer_norm', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the ckeckpoint was trained on, you can already use TFDistilBertModel for predictions without further training.




 THESE ARE THE LAYERS
Model: "functional_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_meta (InputLayer)         [(None, 1, 8)]       0                                            
__________________________________________________________________________________________________
input_idx (InputLayer)          [(None, 150)]        0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        [(None, 150)]        0                                            
__________________________________________________________________________________________________
bidirectional_5 (Bidirectional) (None, 1, 64)        10496       input_meta[0][0]                 
______________________________________________________________